In [2]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import MultipleLocator
import os.path
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from qbo1d import utils
from qbo1d import adsolver
from qbo1d import emulate
from qbo1d.stochastic_forcing import WaveSpectrum

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from qbo1d.stochastic_forcing import sample_sf_cw
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

%load_ext autoreload
%autoreload 2

In [3]:
# data generator:
def data_generator(state=1):
    '''
    Input: state(0~4)
    STATE = 0 -> old control
    STATE = 1 -> new control
    STATE = 2 -> different mean
    STATE = 3 -> different variance
    STATE = 4 -> anti-correlation(non-physical) 

    Output: (u, s, sf, cw, solver)
    '''

    # parameter dicts
    sfe = [3.7e-3, 3.8e-3, 3.2e-3, 3.8e-3, 3.8e-3]
    sfv = [1e-8, 9e-8, 9e-8, 9e-10, 9e-8]
    cwe = [32, 32, 40, 32, 32]
    cwv = [225, 225, 225, 225, 225]
    corr = [0.75, 0.75, 0.75, 0.75, -0.75]
    seed = [int(21*9+8), int(21*9+7), int(21*6+15), int(21*12+5), int(21*2+10)]

    # generate the matrix form

    para_mat = np.array([sfe, sfv, cwe, cwv, corr, seed]).T


    # Load the data manually
    # it takes 40 seconds

    t_max = 360 * 108 * 86400
    nsteps = 360 * 108
    nspinup = 360 * 12
    ntot = int(nsteps - nspinup)

    torch.set_default_dtype(torch.float64)


    # scenario 0 (control)
    # --------------------
    solver = adsolver.ADSolver(t_max=t_max, w=3e-4)
    model = WaveSpectrum(solver, *para_mat[state])
    time = solver.time
    z = solver.z
    u = solver.solve(source_func=model)

    return u, model.s, model.sf, model.cw, solver


In [4]:
STATE = 0
u, s, sf, cw, solver = data_generator(state=STATE)

/Users/eddysmac/Desktop/QBO-1d/qbo1d/adsolver.py:145: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  u[n + 1] = torch.triangular_solve(b, self.R).solution.flatten()


In [5]:
# Data preprocessing
nsteps = 360 * 108
nspinup = 360 * 12


U = u[nspinup:nsteps, :]
SF = sf[nspinup:nsteps]
CW = cw[nspinup:nsteps]

U = torch.hstack([U[:, 1:-1], SF.view(-1, 1), CW.view(-1, 1)])

S = s[nspinup:nsteps, :]

sc_U = StandardScaler()
sc_S = StandardScaler()

# Here U is the features and s is the label

U_train, U_test, s_train, s_test = train_test_split(U, S, test_size=0.8, random_state=42)

U_train = sc_U.fit_transform(U_train)
U_test = sc_U.transform(U_test)

s_train = sc_S.fit_transform(s_train)
s_test = sc_S.transform(s_test)

- The reason why we need scaling: Otherwise we need to set  a really small $\epsilon$ 

In [6]:
s_train_35 = s_train[:, 35]
s_test_35 = s_test[:, 35]

In [7]:
svr = SVR(kernel='rbf', gamma='auto', epsilon=.1, C=32)
svr.fit(U_train, s_train_35)

SVR(C=32, gamma='auto')

In [8]:
print(svr.score(U_test, s_test_35))
print(svr.score(U_train, s_train_35))

0.9243928328457395
0.931846299359639


In [9]:
svr.n_support_

array([1041], dtype=int32)

In [118]:
svr_1d = SVR(kernel='rbf', gamma='auto', epsilon=.1, C=32)
mr = MultiOutputRegressor(svr)
mr.fit(U_train, s_train)

MultiOutputRegressor(estimator=SVR(C=32, gamma='auto'))

In [119]:
mr.score(U_test, s_test)

0.9294644251935146

In [120]:
prediction = mr.predict(U_test)
mean_l2_loss = np.linalg.norm((prediction - s_test), axis=1).mean()
print(f"The loss of the linear regression: {mean_l2_loss}")
# print(f"R-squared: {mr.score(U_test, s_test):.9f}")

The loss of the linear regression: 1.9204686592371831


In [121]:
prediction_ = sc_S.inverse_transform(prediction)
s_test_ = sc_S.inverse_transform(s_test)
mean_l2_loss_ = np.linalg.norm((prediction_ - s_test_), axis=1).mean()
print(f"The loss of the linear regression: {mean_l2_loss_}")


The loss of the linear regression: 2.140788305015839e-05


In [122]:
def rMSE(s_gt, s_pred):
    error = (s_gt - s_pred)
    SSE = sum(error ** 2)
    MSE = SSE/s_gt.shape[0]
    RMSE = MSE**.5
    return RMSE

def plot_MSE(RMSE_list):
    levels = list(range(len(RMSE_list[0])))
    for RMSE in RMSE_list:
        plt.scatter(x=RMSE, y=levels, s=2)
        plt.plot(RMSE, levels)
    plt.show()

In [129]:
transformed_U = sc_U.transform(U)
s_prediction = mr.predict(transformed_U)
s_prediction_ = sc_S.inverse_transform(prediction)
RMSE = rMSE(s, s_prediction_) 

KeyboardInterrupt: 

In [128]:
transformed_U.shape

(34560, 73)